# Pose Control

SPDX-License-Identifier: MIT  
SPDX-FileCopyrightText: 2022 Daisuke Sato

https://github.com/Tiryoh/mini-pupper-jupyter-notebooks

## Reference

* https://github.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi

## Set Network configuration

In [ ]:
# NETWORK_INTERFACE="eth0"
NETWORK_INTERFACE="wlan0"

import netifaces as ni
ip_address = ni.ifaddresses(NETWORK_INTERFACE)[ni.AF_INET][0]['addr']
print(ip_address)

In [ ]:
import os

# Set self ip address
os.environ['ROS_IP'] = ip_address

# Set ROS_MASTER_URI
ROS_MASTER_IP=ip_address
# ROS_MASTER_IP="192.168.10.14" # modify here to specify custom ip

os.environ['ROS_MASTER_URI'] = 'http://{}:11311'.format(ROS_MASTER_IP)
print("ROS_MASTER_IP is {}".format(ROS_MASTER_IP))

### Check environment  

It should outputs like this:

```
192.168.10.12
http://192.168.10.12:11311
```

If the result is `http://localhost:11311`, re-run the above procedure.

In [ ]:
# Check Env
!echo $ROS_IP
!echo $ROS_MASTER_URI

## Import libraries

In [ ]:
from sidecar import Sidecar
# Using jupyros(jupyter-ros) 0.5.0 from conda for now
try:
    from jupyros import ros3d
except ImportError:
    from jupyros.ros1 import ros3d

# Check jupyros version
import jupyros
print(jupyros.__version__)

## Publish pose topic and control robot

### Check topic

If it shows `ERROR: Unable to communicate with master!` or `ERROR: Unknown topic /body_pose`, re-run the network configuration step.

In [ ]:
!rostopic info /body_pose

### Prepare publisher

In [ ]:
import jupyros
import rospy
from sensor_msgs.msg import Image
from geometry_msgs.msg import Pose

rospy.init_node("pose_publisher")
pub = rospy.Publisher('/body_pose', Pose, queue_size=10)

### Create widget

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets
import math

roll = 0
pitch = 0
yaw = 0

yaw_widget = widgets.FloatSlider(min = -0.2, max = 0.2, step = 0.01, description='yaw')
pitch_widget = widgets.FloatSlider(min = -0.2, max = 0.2, step = 0.01, description='pitch')


display(widgets.VBox([
    pitch_widget,
    yaw_widget
]))

#### Enable widget

Run the below cell to start publishing pose topic form widget.

In [ ]:
%%thread_cell

rate = rospy.Rate(5)
while not rospy.is_shutdown():
    pose = Pose()

    yaw = yaw_widget.value
    pitch = pitch_widget.value

    cy = math.cos(yaw)
    sy = math.sin(yaw)
    cp = math.cos(pitch)
    sp = math.sin(pitch)
    cr = math.cos(roll)
    sr = math.sin(roll)

    pose.orientation.w = cy * cp * cr + sy * sp * sr
    pose.orientation.x = cy * cp * sr - sy * sp * cr
    pose.orientation.y = sy * cp * sr + cy * sp * cr
    pose.orientation.z = sy * cp * cr - cy * sp * sr

    pub.publish(pose)
    rate.sleep()

In [ ]:
jupyros.subscribe("/body_pose", Pose, lambda msg: print(msg))